# Hands-on Workshop - *Predicting product ratings from reviews*

This workshop is focused on the creation, deployment, monitoring and management of a machine learning model for predicting product ratings from reviews.

### Tom - this isn't completely relevant anymore, will change it before workshop to talk about pretrained model being used etc, so just ignore for now

In this notebook we will be exploring the data, and training the machine learning model itself; in the form of a Tensorflow Keras model. We will then deploy a second Tensorflow model with differing architecture as a Canary model to demonstrate the A/B testing functionality Seldon provides. Then we will begin to add the advanced monitoring and explainability that Seldon Alibi is famed for. 

-----------------------------------
Firstly, we will install and import the relevant packages which we will use throughout the exploration, training, and deployment process. 

In [ ]:
!pip install transformers
!pip install seldon_deploy_sdk
!pip install alibi==0.6.4
!pip install alibi_detect==0.8.1
!pip install datasets

In [ ]:
import numpy as np
import pandas as pd 
import datasets

import tensorflow as tf
from transformers import AutoTokenizer, create_optimizer, DefaultDataCollator, TFAutoModelForSequenceClassification

from sklearn.model_selection import train_test_split

from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi, ModelMetadataServiceApi, DriftDetectorApi, BatchJobsApi, BatchJobDefinition
from seldon_deploy_sdk.auth import OIDCAuthenticator

In [ ]:
# Check available GPUs

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Get Data

In [ ]:
!gsutil cp gs://kelly-seldon/nlp-ratings/review_data.csv review_data.csv

In [ ]:
df = pd.read_csv("review_data.csv", delimiter=";")
df.head()

## Preprocess Data

Can we do more here? Remove punctuation etc? What does the pretrained tokenizer do?

In [ ]:
df['review'] = df['review'].astype(str)
df['rating'] = df['rating'].astype(str)

rating_mapping = {
    '1.0': 0,
    '1.5': 1,
    '2.0': 2,
    '2.5': 3,
    '3.0': 4,
    '3.5': 5,
    '4.0': 6, 
    '4.5': 7,
    '5.0': 8
}

df['label'] = df['rating'].apply(lambda x: rating_mapping[x])
df.drop(columns=['product', 'user_id', 'date_created', 'rating'], axis=1, inplace=True)

df.head()

In [ ]:
df["label"].value_counts()

In [ ]:
# Split data into train and test sets
train, test = train_test_split(df, test_size=0.3, random_state=42)

In [ ]:
# Convert data into Dataset type
train_ds = datasets.Dataset.from_pandas(train, preserve_index=False)
test_ds = datasets.Dataset.from_pandas(test, preserve_index=False)
comp_ds = datasets.DatasetDict({"train":train_ds,"test":test_ds})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(df):
    return tokenizer(df["review"], padding="max_length", truncation=True)

In [ ]:
tokenized_revs = comp_ds.map(preprocess_function, batched=True)

In [ ]:
data_collator = DefaultDataCollator(return_tensors="tf")

In [ ]:
tf_train_set = tokenized_revs["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator
)

tf_test_set = tokenized_revs["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator
)

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=9)

In [ ]:
model.layers[0].trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(tf_train_set, validation_data=tf_test_set, epochs=5)

In [ ]:
# reviews = []
# labels = []

# for row in df.iterrows():
#     labels.append(row[1]['label'])
#     review = row[1]['review']
#     for word in STOPWORDS:
#         token = ' ' + word + ' '
#         review = review.replace(token, ' ')
#         review = review.replace(' ', ' ')
#     reviews.append(review)
# print(len(labels))
# print(len(reviews))

## Evaluation

In [ ]:
# colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
# mpl.rcParams['figure.figsize'] = (12, 10)

In [ ]:
# def plot_metrics(history):
#     metrics = ['loss', 'accuracy']
#     for n, metric in enumerate(metrics):
#         name = metric.replace("_"," ").capitalize()
#         plt.subplot(2,2,n+1)
#         plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
#         plt.plot(history.epoch, history.history['val_'+metric],
#                  color=colors[0], linestyle="--", label='Val')
#         plt.xlabel('Epoch')
#         plt.ylabel(name)
#         if metric == 'loss':
#             plt.ylim([0, plt.ylim()[1]])
#         elif metric == 'auc':
#             plt.ylim([0.8,1])
#         else:
#             plt.ylim([0,1])

#         plt.legend();

In [ ]:
# plot_metrics(history)

In [ ]:
# int_loss, int_accuracy = int_model.evaluate(int_test_ds[0], int_test_ds[1])

# print("Int model accuracy: {:2.2%}".format(int_accuracy))

In [ ]:
# predictions = int_model.predict(int_test_ds[0])
# print(predictions[:1])

In [ ]:
# print(predictions)

In [ ]:
# test_re = int_test_ds[1].reset_index(drop=True)

In [ ]:
# test_val_list = []
# pred_val_list = []

# for i in range(0,len(test_re)):
#     test_val = test_re[i]
#     test_val_list.append(test_val)
#     pred_val = list(predictions[i]).index(predictions[i].max())
#     pred_val_list.append(pred_val)

In [ ]:
# df_results = pd.DataFrame(
#         {'Test_val':test_val_list,
#             'Pred_val':pred_val_list}

# )

In [ ]:
# df_results

In [ ]:
# df_results['match']='False'
# df_results.loc[df_results['Test_val']==df_results['Pred_val'],'match'] = 'True'

In [ ]:
# df_group = df_results.groupby(['Test_val','match']).count()

In [ ]:
# df_results['Pred_val'].value_counts(normalize=True)

In [ ]:
# df_group

## Evaluation if accuracy not provided as metric when training

In [ ]:
y_preds = model.predict(tf_train_set)

In [ ]:
y_preds["logits"][3000]

In [ ]:
rating_preds = []
for i in y_preds["logits"]:
    rating_preds.append(np.argmax(i, axis=0))

In [ ]:
print('accuracy %s' % accuracy_score(rating_preds, test["label"]))

## Model Deployment

You can now deploy your model to the dedicated Seldon Deploy cluster which we have configured for this workshop. To do so you will interact with the Seldon Deploy SDK and deploy your model using that.

First, setting up the configuration and authentication required to access the cluster. Make sure to fill in the `SD_IP` variable to be the same as the cluster you are using.

In [ ]:
SD_IP = "34.141.146.222"

config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_secret = "sd-api-secret"
config.auth_method = "client_credentials"

def auth():
    auth = OIDCAuthenticator(config)
    config.id_token = auth.authenticate()
    api_client = ApiClient(configuration=config, authenticator=auth)
    return api_client

Now you have configured the IP correctly as well as setup your authentication function you can describe the deployment you would like to create. 

You will need to fill in the `YOUR_NAME` variable. This MUST be lower case as it will be used for the `DEPLOYMENT_NAME` variable later on.

The `MODEL_NAME` and `MODEL_PATH` variables have been prefilled for you as we are using a pretrained model and a pre-built container image for the sake of saving time in the workshop. 

The rest of the `mldeployment` description has been completed for you.


In [ ]:
YOUR_NAME = "kellyspry"
MODEL_NAME = "review-ratings"

DEPLOYMENT_NAME = f"{YOUR_NAME}-{MODEL_NAME}"
CONTAINER_NAME = f"kellyspry0316/{MODEL_NAME}:0.6"

NAMESPACE = "seldon-gitops"

CPU_REQUESTS = "0.1"
MEMORY_REQUESTS = "1Gi"

CPU_LIMITS = "0.1"
MEMORY_LIMITS = "1Gi"

MODEL_PATH = "nlp-ratings/1/"

In [ ]:
mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol": "seldon",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "image": CONTAINER_NAME,
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 100,
                "graph": {
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "parameters": [
                        {
                            "name":"model_path",
                            "value":MODEL_PATH,
                            "type":"STRING"
                        }
                    ],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            }
        ]
    },
    "status": {}
}
      

You can now invoke the SeldonDeploymentsApi and create a new Seldon Deployment.

Time for you to get your hands dirty. You will use the Seldon Deploy SDK to create a new Seldon deployment. You can find the reference documentation [here](https://github.com/SeldonIO/seldon-deploy-sdk/blob/master/python/README.md).

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

Example of a good review:

```
{
    "data": {
                "ndarray": [
                    [
                    "This product is the best, it is so amazing. Incredible!"
                    ]
                 ]
           }
}
                        
```


Example of a bad review:

```
{
    "data": {
                "ndarray": [
                    [
                    "This product is the worst, it is absolutely terrible. Awful!"
                    ]
                 ]
           }
}
                        
```

You can access the Seldon Deploy cluster and view your freshly created deployment here:

### To do - add cluster IP in here when cluster is decided on

* URL: http://34.141.146.222/seldon-deploy/
* Username: admin@seldon.io
* Password: 12341234

## Adding a Prediction Schema and Metadata

Seldon Deploy has a model catalog where all deployed models are automatically registered. The model catalog can store custom metadata as well as prediction schemas for your models.

Metadata promotes lineage from across different machine learning systems, aids knowledge transfer between teams, and allows for faster deployment. Meanwhile, prediction schemas allow Seldon Deploy to automatically profile tabular data into histograms, allowing for filtering on features to explore trends.

In order to effectively construct a prediction schema Seldon has the ML Prediction Schema project. 

In [ ]:
prediction_schema = {
    "requests": [
        {
            "name": "Product Review",
            "type": "TEXT",
        }
    ],
    "responses": [
        {
            "name": "Rating",
            "type": "CATEGORICAL",
            "data_type": "INT",
            "n_categories": 9,
            "category_map": {
            "0": "1.0",
            "1": "1.5",
            "2": "2.0",
            "3": "2.5",
            "4": "3.0",
            "5": "3.5",
            "6": "4.0",
            "7": "4.5",
            "8": "5.0"
        }
        }
    ]
}

You then add the prediction schema to the wider model catalog metadata. This includes information such as the model storage location, the name, who authored the model etc. The metadata tags and metrics which can be associated with a model are freeform and can therefore be determined based upon the use case which is being developed. 

In [ ]:
# To do if time - add accuracy as a metric in here once have a working model

model_catalog_metadata = {
      "URI": MODEL_LOCATION,
      "name": f"{DEPLOYMENT_NAME}-model",
      "version": "v1.0",
      "artifactType": "Tensorflow",
      "taskType": "Product review rating classification",
      "tags": {
        "auto_created": "true",
        "author": f"{YOUR_NAME}"
      },
      "metrics": {
          "accuracy": 0,
      },
      "project": "default",
      "prediction_schema": prediction_schema
    }

model_catalog_metadata

Next, using the metadata API you can add this to the model which you have just created in Seldon.

In [ ]:
metadata_api = ModelMetadataServiceApi(auth())
metadata_api.model_metadata_service_update_model_metadata(model_catalog_metadata)

You can then list the metadata via the API, or view it in the UI, to confirm that it has been successfully added to the model. 

In [ ]:
metadata_response = metadata_api.model_metadata_service_list_model_metadata(uri=MODEL_LOCATION)
metadata_response

## Make Predictions

Now you can have a go at sending some requests to your model using the 'Predict' tab in the UI. 

An example of a good review that we would expect to correspond to a higher rating.

```
{
    "data": {
        "names": ["Review"],
        "ndarray": ["_product_ is excellent! I love it, it's great!"]
    }
}
```

And an example of a negative review that we would expect to correspond to a lower rating.

```
{
    "data": {
        "names": ["Review"],
        "ndarray": ["_product_ was terrible, I would not use it again, it was awful!"]
    }
}
```

## Drift Detection

Although powerful, modern machine learning models can be sensitive. Seemingly subtle changes in a data distribution can destroy the performance of otherwise state-of-the art models, which can be especially problematic when ML models are deployed in production. Typically, ML models are tested on held out data in order to estimate their future performance. Crucially, this assumes that the process underlying the input data `X` and output data `Y` remains constant.

Drift can be classified into the following types:
* **Covariate drift**: Also referred to as input drift, this occurs when the distribution of the input data has shifted `P(X) != Pref(X)`, whilst `P(Y|X) = Pref(Y|X)`. This may result in the model giving unreliable predictions.

* **Prior drift**: Also referred to as label drift, this occurs when the distribution of the outputs has shifted `P(Y) != Pref(Y)`, whilst `P(X|Y) = Pref(X|Y)`. This can affect the model’s decision boundary, as well as the model’s performance metrics.

* **Concept drift**: This occurs when the process generating `Y` from `X` has changed, such that `P(Y|X) != Pref(Y|X)`. It is possible that the model might no longer give a suitable approximation of the true process.

-----------------

In this instance we will train a Kolmgorov-Smirnov drift detector to pick up on covariate drift. The KS Drift detector applies a two-sample KS test to compare the distance between the new probability distribution and the reference distribution. 

This is done on a feature by feature basis and the results are then aggregated using a correction, i.e. Bonferroni, to determine whether drift has occurred overall within the sample. 

We will use the training set as our reference distribution. Creating our drift detector is then as simple as writing a single line of code:

In [ ]:
dd_array = np.asarray(encoded_dataset['train']['input_ids'][:5000])

In [ ]:
dd = KSDrift(dd_array, p_val=.05, correction='bonferroni')

In [ ]:
preds = dd.predict(dd_array)
labels = ["No", "Yes"]

print(f'Drift? {labels[preds["data"]["is_drift"]]}')

In [ ]:
preds = dd.predict(
    np.asarray(encoded_dataset['test']['input_ids'][6700:6800]), 
    return_p_val=True, 
    return_distance=True)
preds

In [ ]:
# Do we need something here to decode before using for explanations?

In [ ]:
save_detector(dd, "reviews-drift-detector")

In [ ]:
## Will add this earlier when saving the model

YOUR_NAME = "kelly-spry"

In [ ]:
!gsutil cp -r reviews-drift-detector gs://kelly-seldon/nlp-ratings/models/{YOUR_NAME}/reviews-drift-detector

To deploy the drift detector you will use Seldon Deploy's user interface. Simply navigate to your deployment, and select the "Create" button for your drift detector.

This will bring up a form. Add your detector name, the URI which will be gs://kelly-seldon/nlp-ratings/models/{YOUR_NAME}/reviews-drift-detector, and set a batch size of 10.

The batch size configuration sets how many data points have to be sent to the endpoint before drift is calculated.

## Run a Batch Job

The simplest way to test that your drift detector and prediction schema are behaving correctly is to kick off a batch job. 

There is already a pre-prepared batch of data stored in a Minio bucket which Seldon can access, therefore all you have to do is provide the configuration which is outlined below. This shows all of the available parameters which can be changed, but not all are required and are shown for educational purposes. 

In [ ]:
# Ask Tom about batch_gateway_endpoint - is this the name of the cluster? is this arbituary? 
# Should this be hard coded or changed to match peoples model names in the workshop

workflow = BatchJobDefinition(
    batch_data_type='data',
    batch_gateway_type='seldon',
    batch_interval=0.0,
    batch_method='predict',
    batch_payload_type='ndarray',
    batch_retries='3',
    batch_size=0,
    batch_transport_protocol='rest',
    batch_workers='1',
    input_data='minio://batch/reviews-batch.txt',
    object_store_secret_name='minio-bucket',
    output_data='minio://batch/reviews-batch-output.txt',
    pvc_size='1Gi'
)

You can then pass the above configuration to the batch job API and kick it off. 

In [ ]:
batch_api = BatchJobsApi(auth())
batch_api.create_seldon_deployment_batch_job(DEPLOYMENT_NAME, NAMESPACE, workflow)

## Explainability

Next, you will train an explainer to glean deeper insights into the decisions being made by your model. 

You will make use of the Anchors algorithm, which has a [production grade implementation available](https://docs.seldon.io/projects/alibi/en/stable/methods/Anchors.html) using the Seldon Alibi Explain library. 

The first step will be to write a simple prediction function which the explainer can call in order to query your Tensorflow model. 

In [ ]:
def predict_fn(x):
    return clf.predict_proba(x)

You then initialise your Anchor explainer, using the AnchorText flavour provided by Alibi due to your data modality. 

### This text might need changing based on what we use as the language model for this. One to discuss with Tom.

The AnchorText class expects the prediction function which you defined above, as well as a sampling strategy and nlp/language model. You can find a sample notebook in the Alibi docs [here]https://docs.seldon.io/projects/alibi/en/stable/examples/anchor_text_movie.html). 

Load spacy model

In [ ]:
model = 'en_core_web_md'
spacy_model(model=model)
nlp = spacy.load(model)

In [ ]:
explainer = AnchorText(
    predictor=predict_fn,
    sampling_strategy='similarity',   # Replace masked words by simialar words
    nlp=nlp,                          # Spacy object
    sample_proba=0.5,                 # Probability of a word to be masked and replace by as similar word
)

You can then run an explanation for a given review locally.

In [ ]:
explanation = explainer.explain(text, threshold=0.9)

You now save your explainer, and upload it to the GS bucket. You can use the explainer's built-in save method to do this easily and reproducibly. 

In [ ]:
explainer.save("review-explainer")

In [ ]:
!gsutil cp -r reviews-explainer gs://kelly-seldon/nlp-ratings/models/{YOUR_NAME}/reviews-explainer

## Deploying the Explainer

You can now deploy our explainer alongside our model. First you define the explainer configuration. 

In [ ]:
EXPLAINER_TYPE = "AnchorText"
EXPLAINER_URI = f"gs://kelly-seldon/nlp-ratings/models/{YOUR_NAME}/reviews-explainer"

explainer_spec = {
                    "type": EXPLAINER_TYPE,
                    "modelUri": EXPLAINER_URI,
                    "containerSpec": {
                        "name": "",
                        "resources": {}
                    }
                }

You can then insert this additional configuration into your original `mldeployment` specification which you defined earlier. 

In [ ]:
mldeployment['spec']['predictors'][0]['explainer'] = explainer_spec
mldeployment

You can then deploy the explainer to the Seldon Deploy cluster!

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

You can then run the same predictions through your model as above and generate explanations on them:

An example of a good review that we would expect to correspond to a higher rating.

```
{
    "data": {
        "names": ["Review"],
        "ndarray": ["_product_ is excellent! I love it, it's great!"]
    }
}
```

And an example of a negative review that we would expect to correspond to a lower rating.

```
{
    "data": {
        "names": ["Review"],
        "ndarray": ["_product_ was terrible, I would not use it again, it was awful!"]
    }
}
```